In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
from matplotlib.ticker import StrMethodFormatter
from matplotlib import rcParams
from matplotlib.pyplot import figure

In [ ]:
social_df = pd.read_excel('../data/social_criteria.xlsx').dropna()
env_criteria = pd.read_excel('../data/env_cri.xlsx', sheet_name="Form Responses 1").dropna()